In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request, json
import re
import matplotlib as plt
import sqlalchemy
import sqlite3

## WATCH OUT, WE'RE LOADING DATA HERE

In [2]:
url_endpoint = 'https://data.cityofchicago.org/resource/cwig-ma7x.json'
response = requests.get(url_endpoint)
data = response.json()
df = pd.DataFrame(data)
df.violations = df.violations.fillna('')


In [69]:
df.violations[1]

'30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABELED: CUSTOMER ADVISORY POSTED AS NEEDED - Comments: NO LABEL PROVIDED AT INDIVIDUAL RE-PACKED SLICE OF CHEESE CAKE SOLD TO CUSTOMERS.INSTRUCTED TO PROVIDE SOURCE AND INGREDIENTS ON EACH CONTAINER.ALSO ANY FOOD OR NON-FOOD PRODUCTS REMOVED FROM ORIGINAL CONTAINERS MUST HAVE A LABEL | 34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOOD REPAIR, COVING INSTALLED, DUST-LESS CLEANING METHODS USED - Comments: FLOOR DRAIN UNDER THE THREE COMPARTMENT SINK WITH SLIME BUILD-UP. DEBRIS UNDER SHELVES,COOLERS AND BEHIND REACH-IN COOLER.INSTRUCTED TO CLEAN AND MAINTAIN | 32. FOOD AND NON-FOOD CONTACT SURFACES PROPERLY DESIGNED, CONSTRUCTED AND MAINTAINED - Comments: PLASTIC  CONTAINERS INSIDE DRY CHEESE ,INSTRUCTED TO REMOVE AND PROVIDED SCOOP WITH HANDLES | 41. PREMISES MAINTAINED FREE OF LITTER, UNNECESSARY ARTICLES, CLEANING  EQUIPMENT PROPERLY STORED - Comments: ALL PERSONAL ITEMS MUST BE REMOVED FROM FOOD AND FOOD AREA.'

**Create a list of infractions for each Inspection**

In [60]:
i=0
l=[]
while i < len(df):
    s = df['violations'][i]
    t1 = re.findall("\d+",str(re.findall("\|\s\d+",s)))
    st1 = ','.join(t1)
    l.append(st1)
    i=i+1

len(l)
df['infracs']=l

In [57]:
#TEST CELL
s = df['violations'][1]
t1 = re.findall("\d+",str(re.findall("\|\s\d+",s)))
st1 = ','.join(t1)
st1

'34,32,41'

In [77]:
critical = range(1,15)
serious = range(15,30)
minor = range(30,70)


## DATABASE PARTY ZONE

In [65]:
#Prep DataFrame for SQLite
l = range(5,len(df.columns))
testdf = df.iloc[:,l]
testdf['location'] = testdf['location'].astype(str)
testdf.head()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip,infracs
0,1763 W HOWARD ST,JEWEL FOOD STORE,CHICAGO,JEWEL FOOD STORE,Grocery Store,2018-02-16T00:00:00.000,2145045,Complaint Re-Inspection,42.01928855008811,2320250,"{'type': 'Point', 'coordinates': [-87.67505850...",-87.67505850938775,Pass,Risk 1 (High),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60626,"26,32,33,34,35,38,40"
1,1435 W MONTROSE AVE,SARPINO'S PIZZERIA,CHICAGO,"WRIGLEYVILLE PIZZERIA, INC.",Restaurant,2018-02-16T00:00:00.000,2145086,Complaint,41.96152004287502,2511423,"{'type': 'Point', 'coordinates': [-87.66634370...",-87.66634370710501,Pass,Risk 2 (Medium),IL,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",60613,"34,32,41"
2,4570 N BROADWAY,PALLAS GRILL,CHICAGO,PALLAS GRILL,Restaurant,2018-02-16T00:00:00.000,2145077,Canvass Re-Inspection,41.96527014438852,2196649,"{'type': 'Point', 'coordinates': [-87.65755429...",-87.6575542920256,Pass,Risk 1 (High),IL,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,60640,"32,33,35,36,38"
3,1320 W DEVON AVE,THAI SPICE,CHICAGO,THAI SPICE,Restaurant,2018-02-16T00:00:00.000,2145066,Canvass,41.99826122453937,38123,"{'type': 'Point', 'coordinates': [-87.66394414...",-87.6639441429001,Out of Business,Risk 1 (High),IL,,60660,
4,2918-2920 N Clark ST,Yuki Hana,CHICAGO,Yuki Hana,Restaurant,2018-02-16T00:00:00.000,2145096,Complaint,41.93528683052736,2288846,"{'type': 'Point', 'coordinates': [-87.64707246...",-87.64707246891297,Pass w/ Conditions,Risk 1 (High),IL,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,60657,"21,38,34,35"


In [66]:
#Create DB, load Panda DF
conn = sqlite3.connect("FoodInspect.db")
c = conn.cursor()
testdf.to_sql("Inspections",conn, if_exists="replace")
t = pd.read_sql_query("select * from Inspections limit 5;", conn)
conn.close()

In [67]:
t

,index,address,aka_name,city,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip,infracs
0,0,1763 W HOWARD ST,JEWEL FOOD STORE,CHICAGO,JEWEL FOOD STORE,Grocery Store,2018-02-16T00:00:00.000,2145045,Complaint Re-Inspection,42.01928855008811,2320250,"{'type': 'Point', 'coordinates': [-87.67505850...",-87.67505850938775,Pass,Risk 1 (High),IL,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,60626,"26,32,33,34,35,38,40"
1,1,1435 W MONTROSE AVE,SARPINO'S PIZZERIA,CHICAGO,"WRIGLEYVILLE PIZZERIA, INC.",Restaurant,2018-02-16T00:00:00.000,2145086,Complaint,41.96152004287502,2511423,"{'type': 'Point', 'coordinates': [-87.66634370...",-87.66634370710501,Pass,Risk 2 (Medium),IL,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",60613,"34,32,41"
2,2,4570 N BROADWAY,PALLAS GRILL,CHICAGO,PALLAS GRILL,Restaurant,2018-02-16T00:00:00.000,2145077,Canvass Re-Inspection,41.96527014438852,2196649,"{'type': 'Point', 'coordinates': [-87.65755429...",-87.6575542920256,Pass,Risk 1 (High),IL,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,60640,"32,33,35,36,38"
3,3,1320 W DEVON AVE,THAI SPICE,CHICAGO,THAI SPICE,Restaurant,2018-02-16T00:00:00.000,2145066,Canvass,41.99826122453937,38123,"{'type': 'Point', 'coordinates': [-87.66394414...",-87.6639441429001,Out of Business,Risk 1 (High),IL,,60660,
4,4,2918-2920 N Clark ST,Yuki Hana,CHICAGO,Yuki Hana,Restaurant,2018-02-16T00:00:00.000,2145096,Complaint,41.93528683052736,2288846,"{'type': 'Point', 'coordinates': [-87.64707246...",-87.64707246891297,Pass w/ Conditions,Risk 1 (High),IL,8. SANITIZING RINSE FOR EQUIPMENT AND UTENSILS...,60657,"21,38,34,35"
